In [0]:
! pip install -q tensorflow-gpu==2.0.0-rc0
import tensorflow as tf
print(tf.__version__)

- Getting the optimal model requires tuning many interdependant performances
- Keras Tuner: Hypertuning for humans

In [0]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten

In [0]:
import numpy as np

In [0]:
model = tf.keras.models.Sequential()
model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform'))
model.add(Flatten())
model.add(Dense(20))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

In [0]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
X_train = tf.cast(np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)), tf.float64)
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)), tf.float64)

In [0]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [0]:
model.fit(X_train, y_train, epochs=10, batch_size=128)

## Features of Keras-Tuner
- **Intuitive API**: As easy as 1,2,3
- **State of the art hypertuner algorithms**
- **Tunable architectures ready to go**
- **Seamless experiments recording**: Automatic recording to analyse and reproduce your results

In [0]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [0]:
X_train = tf.cast(np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)), tf.float64)
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)), tf.float64)

In [0]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [0]:
# Step 1: Wrap model in a function
def model_fn(hp):
    
    # Step 2: Define the hyper-parameters
    LR = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
    DROPOUT_RATE = hp.Float('dropout_rate', 0.0, 0.5, 5)
    NUM_DIMS = hp.Int('num_dims',  8, 32, 8)
    NUM_LAYERS = hp.Int('num_layers', 1, 3)
    L2_NUM_FILTERS = hp.Int('l2_num_filters', 8, 64, 8)
    L1_NUM_FILTERS = hp.Int('l1_num_filters', 8, 64, 8)
        
    # Step 3: Replace static values with hyper-parameters
    model = tf.keras.models.Sequential()
    model.add(Conv2D(L1_NUM_FILTERS, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
    model.add(Conv2D(L2_NUM_FILTERS, (3,3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Flatten())
    for _ in range(NUM_LAYERS):
        model.add(Dense(NUM_DIMS))
        model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
    return model
    

In [0]:
!git clone https://github.com/keras-team/keras-tuner.git
!cd keras-tuner
!pip install -q '/content/keras-tuner/'

In [0]:
import kerastuner
from kerastuner.tuners import RandomSearch

In [0]:
tuner = RandomSearch(
    model_fn,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='temp_dir')

In [0]:
tuner.search_space_summary()

In [0]:
tuner.search(X_train, y_train,
                epochs=10,
                validation_data=(X_test, y_test))

In [0]:
models = tuner.get_best_models(num_models=3)

In [0]:
tuner.results_summary()

## Upcoming Features
- **Ecosystem intergration**
- **Online Dashboard**: Monitoring on the go mobile app